# FastText

In [ ]:
# import packages
import numpy as np
import pandas as pd

# model training
import fasttext
from sklearn.metrics import classification_report

# for display
from IPython.display import display, HTML

## Load Data

In [ ]:
# define path and filenames
path = 'data/emoticon/' # 'data/emoticon/'
train = 'train'
val = 'val'
test = 'test'
suffix = '_newpreproc_emoticon.csv'

In [ ]:
# create data dict
data = {}
data_names = [train, val, test]
# change this accordingly: 'phrase', 'phrase_lemma', 'phrase_stem'
# text_column = 'phrase_emoticon_unique'
text_column = 'phrase_emoticon_unique'
old_new = 'new_preproc_emoticon'

for name in data_names:
    # read data
    df = pd.read_csv(path+name+suffix)
    df['label'] = df['label'].astype('int32')
    # add to data dict
    data[f'{name}_{text_column}'] = df

In [ ]:
for df_name, df in data.items():
    print(df_name)
    print(df.shape)
    print(df.label.value_counts())
    display(df.head(3))
    print('\n')

In [ ]:
# combine train and val
train_all = pd.concat([data['train_'+text_column], data['val_'+text_column]])
data['train_all_'+text_column] = train_all

In [ ]:
# convert into txt file for training
train_all_txt = train_all[[text_column, 'label']]

with open(f"data/fasttext/{old_new}/train_all_{text_column}.txt", "w") as f:
    for i in range(len(train_all_txt)):
        row = train_all_txt.iloc[i]
        label = row['label']
        label_txt = ''
        if label == 1:
            label_txt = 'pos'
        elif label == -1:
            label_txt = 'neg'
        elif label == 0:
            label_txt = 'zer'
        line = f'__label__{label_txt} {row[text_column]} \n'
        f.write(line)

## Custom Model

Useful Links:
- https://fasttext.cc/docs/en/supervised-tutorial.html
- https://pypi.org/project/fasttext/
- https://towardsdatascience.com/natural-language-processing-with-fasttext-part-1-an-intro-to-text-classification-with-fasttext-11b9771722d8
- https://fasttext.cc/docs/en/crawl-vectors.html

In [ ]:
model = fasttext.train_supervised(input=f"data/fasttext/{old_new}/train_all_{text_column}.txt")

## Predict on Test

In [ ]:
# predict on test
test_pred_df = data['test_'+text_column].copy()
test_pred_df['raw_output'] = test_pred_df.apply(lambda x: model.predict(x[text_column].replace("\n", "")), axis=1)
test_pred_df['raw_pred'] = test_pred_df.apply(lambda x: x.raw_output[0][0][-3:], axis=1)
test_pred_df['raw_prob'] = test_pred_df.apply(lambda x: x.raw_output[1][0], axis=1)

In [ ]:
test_pred_df.raw_pred.value_counts()

In [ ]:
def label_from_txt(x):
    if x == 'pos':
        return 1
    elif x == 'neg':
        return -1
    else:
        return 0

In [ ]:
test_pred_df['pred'] = test_pred_df.apply(lambda x: label_from_txt(x['raw_pred']), axis=1)

In [ ]:
test_results = classification_report(test_pred_df.label, test_pred_df.pred, output_dict=True)
# save to txt
f = open(f'model_results/fasttext/custom/{old_new}/test_{text_column}.txt', "w")
f.write( str(test_results) )
f.close()

print(classification_report(test_pred_df.label, test_pred_df.pred, digits=4))

In [ ]:
test_pred_df.label.value_counts()

In [ ]:
test_pred_df.pred.value_counts()